<a href="https://colab.research.google.com/github/fjadidi2001/DiabetPrediction/blob/main/Diabet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("himanshu86503/dibetes33hi")

print("Path to dataset files:", path)

100%|██████████| 8.92k/8.92k [00:00<00:00, 2.82MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/himanshu86503/dibetes33hi/versions/1
